# Data

> Setup the data

In [ ]:
#| default_exp data

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import yaml
import os

Carregando .yaml

In [ ]:
#| export
folder_path = "../point_clouds/semantic_kitti/"

with open(folder_path+"semantic-kitti.yaml", 'r') as file:
    data = yaml.safe_load(file)

label_map = data.get('labels', {})
learning_map = data.get('learning_map', {})
color_map = data.get('color_map', {})

Carrega todos os frames e labels de uma dada sequência, e concatena em um único array.
* Args:
    * sequence_path: o caminho para a pasta contendo as pastas "velodyne" e "labels" de uma dada sequência.
* Return:
    * point_cloud: array shape (N,4), onde N é o número de pontos, e as e as dimensões são respectivamente X, Y, Z reflectância e labels.

In [ ]:
#| export
def load_data(sequence_path):
    velodyne_path = os.path.join(sequence_path, 'velodyne')
    labels_path = os.path.join(sequence_path, 'labels')

    frames = []
    for frame_name in sorted(os.listdir(velodyne_path)):
        frame_path = os.path.join(velodyne_path, frame_name)
        with open(frame_path, 'rb') as f:
            frames.append(np.fromfile(f, dtype=np.float32).reshape(-1, 4))

    labels = []
    for label_name in sorted(os.listdir(labels_path)):
        label_path = os.path.join(labels_path, label_name)
        with open(label_path, 'rb') as f:
            np_label = np.fromfile(f, dtype=np.uint32)
            np_label = np_label & 0xFFFF
            labels.append(np_label)

    point_cloud = []
    for frame, label in zip(frames,labels):
        point_cloud.append(np.concatenate((frame, label.reshape(-1, 1)), axis=1))
    
    return point_cloud

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()